# Analysis of intraday storage

Using the base case hydrology and P&E modelling, determine the maximum volume of water that can be held back during two offpeak periods and then released during morning and evening peak hour periods. 

Calculate the energy transferred from the storage (off-peak) to peak periods.

Calculate the rise in the headpond and tailrace during each storage period (and therefore fall during each peak period). 

### Setup

In [233]:
import numpy as np
import pandas as pd
import datetime
import math

In [234]:
def tailwaterLevel(flow):
    if fixed_tailwater_level>0:
        return fixed_tailwater_level
    else:
        log_flow=math.log10(flow)
        if log_flow>=3.8:
            return -21.429*log_flow**2+182.03*log_flow+597.15   
        elif log_flow>=2.3:
            return 6.5299*log_flow**2-30.155*log_flow+999.52   
        else:
            return -2.6828*log_flow**2+11.265*log_flow+952.68

In [562]:
flow_data='../output_data/ngonye_flow_monthly.csv'
energy_data='../output_data/base_pe_monthly.csv'
dam_data='dam_calmonthly.csv'

canal_capacity=1100
rated_flow_unit=250

#offpeak_min_flow=rated_flow_unit
#offpeak_min_flow=rated_flow_unit * 0.5
offpeak_min_flow=50

storage_curve=10 # 10 Mm3/m headpond rise - From Feasibility Study Rev D Appendix H - Mott MacDonald
#max_power=185 # Max output for whole plant
max_power=205
fixed_tailwater_level = 0

night_hours = 10 # Night off-peak
morning_hours = 3 # Length of morning peak
standard_hours = 9 # Time between morning peak and evening peak when storing can occur
evening_hours = 2 # Length of evening peak

nostore=False

### Load data

In [563]:
dam = pd.read_csv(dam_data).set_index('month')
dam.columns

Index(['Unnamed: 0', 'day_of_week', 'value_mean', 'price_weight_mean',
       'price_max', 'price_min', 'price_mean', 'power_max', 'power_min',
       'power_mean', 'price_morning_max', 'price_evening_max',
       'price_standard', 'off_price_mean', 'standard_price_mean',
       'morning_price_mean', 'evening_price_mean', 'off_hour_mean',
       'standard_hour_mean', 'morning_hour_mean', 'evening_hour_mean',
       'off_value', 'standard_value', 'morning_value', 'evening_value',
       'volume', 'off_volume', 'standard_volume', 'morning_volume',
       'evening_volume', 'value'],
      dtype='object')

In [564]:
pe = pd.read_csv(energy_data).set_index('MonthId')
pe.columns

Index(['Flow', 'Exceedance', 'Year', 'Month', 'WaterYear', 'WaterMonth',
       'Flow_difference', 'Flow_difference_pct', 'EWRChannelA', 'EWRChannelC',
       'EWRChannelD', 'EWRChannelE', 'EWRChannelFG', 'EWRTotal',
       'EWRProportion', 'FlowAvailableForGeneration', 'FlowCanal',
       'LowFlowShutoff', 'Turbines', 'FlowTurbine1', 'FlowTurbine2',
       'FlowTurbine3', 'FlowTurbine4', 'FlowSpill', 'SpillChannelA',
       'SpillChannelC', 'SpillChannelD', 'SpillChannelE', 'SpillChannelFG',
       'FlowChannelA', 'FlowChannelC', 'FlowChannelD', 'FlowChannelE',
       'FlowChannelFG', 'FlowLeftChannel', 'LevelTailwater', 'LevelHeadpond',
       'HeadlossLeftChannel', 'HeadlossCanal', 'LevelForebay',
       'HeadlossTurbine1', 'HeadlossTurbine2', 'HeadlossTurbine3',
       'HeadlossTurbine4', 'HeadTurbine1', 'HeadTurbine2', 'HeadTurbine3',
       'HeadTurbine4', 'LowHeadShutoff', 'LoadFactorTurbine1',
       'LoadFactorTurbine2', 'LoadFactorTurbine3', 'LoadFactorTurbine4',
       'EffT

In [565]:
effective_efficiency = 0.9211999999999999

In [566]:
tmp=pe[['Year','Month','WaterYear','WaterMonth','Days','LowHeadShutoff','Flow','EWRTotal','FlowSpill','FlowAvailableForGeneration','FlowCanal','Power','Energy','LevelHeadpond','LevelTailwater', 'HeadTurbine1']].copy()

tmp['LowHeadShutoff'] = tmp['LowHeadShutoff'].fillna(0)
work=tmp
work.columns

Index(['Year', 'Month', 'WaterYear', 'WaterMonth', 'Days', 'LowHeadShutoff',
       'Flow', 'EWRTotal', 'FlowSpill', 'FlowAvailableForGeneration',
       'FlowCanal', 'Power', 'Energy', 'LevelHeadpond', 'LevelTailwater',
       'HeadTurbine1'],
      dtype='object')

### Peak Periods Requirements for maximum output

In [567]:
#Total generation volume Mm3
work['GenerationVol'] = work['FlowCanal'] * (24 * 60 * 60) / 1e+06

#Rough calculation of the maximum flow during peak time - flow that produced maximum output at current head and high efficiency
work['MaxGenerationFlow'] = max_power / work['HeadTurbine1'] / 9.81 / effective_efficiency * 1000
#Limit maximum flow to canal capacity
work['MaxGenerationFlow'] = np.minimum(work['MaxGenerationFlow'],canal_capacity)

#Maximum flow that could be used by the peaks from storage m3/s
work['MaxFlowFromStore'] = work['MaxGenerationFlow'] - work['FlowCanal']

#Max volume that could be used by the peaks Mm3
work['MorningMaxVolFromStore'] = work['MaxFlowFromStore'] * (morning_hours * 60 * 60) / 1e+06
work['EveningMaxVolFromStore'] = work['MaxFlowFromStore'] * (evening_hours * 60 * 60) / 1e+06


### Max volume which the off-peak (night) and standard (day) periods can store

In [568]:
if (nostore):
    #Max volume that could be taken from the night period for storage Mm3
    work['NightMaxVolToStore'] = 0

    #Max volume that could be taken from the standard period for storage Mm3
    work['StandardMaxVolToStore'] = 0

else:  
    #Max volume that could be taken from the night period for storage Mm3
    work['NightMaxVolToStore'] = ((work['FlowAvailableForGeneration'] - offpeak_min_flow) * (night_hours * 60 * 60) / 1e+06).clip(lower=0)

    #Max volume that could be taken from the standard period for storage Mm3
    work['StandardMaxVolToStore'] = ((work['FlowAvailableForGeneration'] - offpeak_min_flow) * (standard_hours * 60 * 60) / 1e+06).clip(lower=0)

    

### Storage volumes

In [569]:
#Volume per night that will be stored (min of available or needed)
work['NightStoreVol'] = work[['MorningMaxVolFromStore','NightMaxVolToStore']].min(axis=1)

#Not enough head for generation so no point storing
work.loc[work['LowHeadShutoff'] == 1.0, 'NightStoreVol'] = 0

#Volume per standard period that will be stored (min of available or needed)
work['StandardStoreVol'] = work[['EveningMaxVolFromStore','StandardMaxVolToStore']].min(axis=1)

#Not enough head for generation so no point storing
work.loc[work['LowHeadShutoff'] == 1.0, 'NightStoreVol'] = 0


### Flows

In [570]:
#Night
work['NightExtraFlow']=(work['NightStoreVol'] / (60 * 60 * night_hours / 1e+06)) # Flow diverted to storage during night
work['NightFlowCanal']=work['FlowCanal'] - work['NightExtraFlow'] #Resulting canal flow during off-peak

#Morning
work['MorningExtraFlow']=(work['NightStoreVol'] / (60 * 60 * morning_hours / 1e+06)) # Extra flow from storage during morning peak 
work['MorningFlowCanal']=work['FlowCanal'] + work['MorningExtraFlow'] #Resulting canal flow during morning peak

#Standard
work['StandardExtraFlow']=(work['StandardStoreVol'] / (60 * 60 * standard_hours / 1e+06)) # Flow diverted to storage during standard period
work['StandardFlowCanal']=work['FlowCanal'] - work['StandardExtraFlow'] #Resulting canal flow during standard period

#Evening
work['EveningExtraFlow']=(work['StandardStoreVol'] / (60 * 60 * evening_hours / 1e+06)) # Extra flow from storage during evening peak 
work['EveningFlowCanal']=work['FlowCanal'] + work['EveningExtraFlow'] #Resulting canal flow during evening peak

### Total flow

In [571]:
work['NightTotalFlow']=work['FlowSpill'] + work['EWRTotal'] + work['NightFlowCanal']
work['MorningTotalFlow']=work['FlowSpill'] + work['EWRTotal'] + work['MorningFlowCanal']
work['StandardTotalFlow']=work['FlowSpill'] + work['EWRTotal'] + work['StandardFlowCanal']
work['EveningTotalFlow']=work['FlowSpill'] + work['EWRTotal'] + work['EveningFlowCanal']

### Total volumes

In [572]:
work['NightVol']=work['NightFlowCanal'] * night_hours * 60 * 60 / 1e+06
work['MorningVol']=work['MorningFlowCanal'] * morning_hours * 60 * 60 / 1e+06
work['StandardVol']=work['StandardFlowCanal'] * standard_hours * 60 * 60 / 1e+06
work['EveningVol']=work['EveningFlowCanal'] * evening_hours * 60 * 60 / 1e+06

### Energy

In [573]:
work['NightEnergy'] = work['Energy'] / work['GenerationVol'] * work['NightVol']
work['MorningEnergy'] = work['Energy'] / work['GenerationVol'] * work['MorningVol']
work['StandardEnergy'] = work['Energy'] / work['GenerationVol'] * work['StandardVol']
work['EveningEnergy'] = work['Energy'] / work['GenerationVol'] * work['EveningVol']

work['MorningEnergyTransfer'] = work['Energy'] / work['GenerationVol'] * work['NightStoreVol']
work['EveningEnergyTransfer'] = work['Energy'] / work['GenerationVol'] * work['StandardStoreVol']


### Power

In [574]:
work['NightPower'] = work['NightEnergy'] / night_hours
work['MorningPower'] = work['MorningEnergy'] / morning_hours
work['StandardPower'] = work['StandardEnergy'] / standard_hours
work['EveningPower'] = work['EveningEnergy'] / evening_hours

### Headpond levels

In [575]:
work['NightHeadpondChange']=work['NightStoreVol'] / storage_curve
work['StandardHeadpondChange']=work['StandardStoreVol'] / storage_curve

### Tailwater levels

In [576]:
work['NightTailwaterLevel']=work['NightTotalFlow'].apply(tailwaterLevel) 
work['MorningTailwaterLevel']=work['MorningTotalFlow'].apply(tailwaterLevel) 
work['StandardTailwaterLevel']=work['StandardTotalFlow'].apply(tailwaterLevel) 
work['EveningTailwaterLevel']=work['EveningTotalFlow'].apply(tailwaterLevel) 

work['NightTailwaterChange']=work['MorningTailwaterLevel'] - work['NightTailwaterLevel']
work['StandardTailwaterChange']=work['EveningTailwaterLevel'] - work['StandardTailwaterLevel']



In [577]:

work.loc[work['MorningMaxVolFromStore'] < work['NightMaxVolToStore'],'NightSupplyLimit'] = 'False'
work.loc[work['MorningMaxVolFromStore'] > work['NightMaxVolToStore'],'NightSupplyLimit'] = 'True'
work.loc[work['EveningMaxVolFromStore'] < work['StandardMaxVolToStore'],'StandardSupplyLimit'] = 'False'
work.loc[work['EveningMaxVolFromStore'] > work['StandardMaxVolToStore'],'StandardSupplyLimit'] = 'True'

work.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 1116 entries, 1924.1 to 2017.09
Data columns (total 61 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Year                        1116 non-null   float64
 1   Month                       1116 non-null   float64
 2   WaterYear                   1116 non-null   float64
 3   WaterMonth                  1116 non-null   float64
 4   Days                        1116 non-null   int64  
 5   LowHeadShutoff              1116 non-null   float64
 6   Flow                        1116 non-null   float64
 7   EWRTotal                    1116 non-null   float64
 8   FlowSpill                   1116 non-null   float64
 9   FlowAvailableForGeneration  1116 non-null   float64
 10  FlowCanal                   1116 non-null   float64
 11  Power                       1116 non-null   float64
 12  Energy                      1116 non-null   float64
 13  LevelHeadpond          

In [578]:
work['NightValue']=work['NightEnergy']*work.join(dam, on='Month')['off_price_mean']/1000000
work['MorningValue']=work['MorningEnergy']*work.join(dam, on='Month')['morning_price_mean']/1000000
work['StandardValue']=work['StandardEnergy']*work.join(dam, on='Month')['standard_price_mean']/1000000
work['EveningValue']=work['EveningEnergy']*work.join(dam, on='Month')['evening_price_mean']/1000000

work['Value']=work['NightValue'] + work['MorningValue'] + work['StandardValue'] + work['EveningValue']
work['Price']=work['Value']/work['Energy']*1000000

work[['Value','Energy','Price']]

,Value,Energy,Price
MonthId,,,
1924.10,0.402405,6668.218097,60.346717
1924.11,1.448394,15948.234986,90.818466
1924.12,3.204916,35452.577303,90.400088
1925.01,5.672401,87660.499151,64.708748
1925.02,7.681795,122211.453841,62.856591
...,...,...,...
2017.05,7.266267,119278.435042,60.918527
2017.06,9.036459,112359.569599,80.424468
2017.07,5.435213,46774.689331,116.199872


In [579]:
calmonths=work.reset_index()[[
    'Month','Energy','Value','Price',
    'NightEnergy','MorningEnergy','StandardEnergy','EveningEnergy',
    'NightValue','MorningValue','StandardValue','EveningValue',
    'MorningEnergyTransfer','EveningEnergyTransfer']].groupby('Month').mean()

calmonths

,Energy,Value,Price,NightEnergy,MorningEnergy,StandardEnergy,EveningEnergy,NightValue,MorningValue,StandardValue,EveningValue,MorningEnergyTransfer,EveningEnergyTransfer
Month,,,,,,,,,,,,,
1.0,83903.037082,5.477370,66.075581,26754.720697,18692.757723,25993.720181,12461.838482,0.951427,1.433160,1.880035,1.212749,8204.878087,5469.918725
2.0,98181.956051,6.346046,65.260154,37176.778158,16005.114703,34272.058535,10728.004655,1.321842,1.292657,2.597286,1.134261,3732.370196,2546.174984
3.0,103752.601623,6.028586,58.755111,42449.782278,14360.405839,38161.655389,9908.503787,1.359032,1.103973,2.575610,0.989971,1250.362427,1168.474846
4.0,98323.751796,5.753204,58.398807,41063.781134,12773.816657,36883.207253,8671.683186,1.289046,0.995064,2.558025,0.911069,349.755628,388.975833
5.0,114093.640328,7.007945,61.516540,46815.922399,14984.799446,42268.860631,10024.057853,1.534965,1.259185,3.084710,1.129085,723.094405,516.254493
6.0,105046.469668,8.496535,82.726720,39770.063796,17130.107274,36677.070884,11469.227714,1.585857,2.085050,3.226806,1.598823,3999.298566,2715.355242
7.0,60456.620121,6.297138,107.602623,14495.348514,18251.987385,15387.297859,12321.986363,0.625552,2.414655,1.450934,1.805998,10694.909870,7283.934686
8.0,36940.203080,4.403136,120.321678,4688.216472,15321.060196,5380.230181,11550.696231,0.189499,1.989927,0.497124,1.726586,10703.534811,8472.345974
9.0,26741.073843,2.677708,99.846600,3452.905479,11031.842852,3465.254782,8791.070729,0.127612,1.080766,0.272550,1.196780,7689.208622,6562.647909


In [580]:
#months.to_csv('../intraday/intraday_monthly_nostore.csv')
#months.to_csv('../intraday/intraday_monthly_rated.csv')
#months.to_csv('../intraday/intraday_monthly_halfrated.csv')
#months.to_csv('../intraday/intraday_monthly_50cumec.csv')
#months.to_csv('../intraday/intraday_monthly_50cumec_205mw.csv')

#calmonths.to_csv('../intraday/intraday_calmonthly_nostore.csv')
#calmonths.to_csv('../intraday/intraday_calmonthly_rated.csv')
#calmonths.to_csv('../intraday/intraday_calmonthly_halfrated.csv')
#calmonths.to_csv('../intraday/intraday_calmonthly_50cumec.csv')
#months.to_csv('../intraday/intraday_calmonthly_50cumec_205mw.csv')

In [581]:
calmonths

,Energy,Value,Price,NightEnergy,MorningEnergy,StandardEnergy,EveningEnergy,NightValue,MorningValue,StandardValue,EveningValue,MorningEnergyTransfer,EveningEnergyTransfer
Month,,,,,,,,,,,,,
1.0,83903.037082,5.477370,66.075581,26754.720697,18692.757723,25993.720181,12461.838482,0.951427,1.433160,1.880035,1.212749,8204.878087,5469.918725
2.0,98181.956051,6.346046,65.260154,37176.778158,16005.114703,34272.058535,10728.004655,1.321842,1.292657,2.597286,1.134261,3732.370196,2546.174984
3.0,103752.601623,6.028586,58.755111,42449.782278,14360.405839,38161.655389,9908.503787,1.359032,1.103973,2.575610,0.989971,1250.362427,1168.474846
4.0,98323.751796,5.753204,58.398807,41063.781134,12773.816657,36883.207253,8671.683186,1.289046,0.995064,2.558025,0.911069,349.755628,388.975833
5.0,114093.640328,7.007945,61.516540,46815.922399,14984.799446,42268.860631,10024.057853,1.534965,1.259185,3.084710,1.129085,723.094405,516.254493
6.0,105046.469668,8.496535,82.726720,39770.063796,17130.107274,36677.070884,11469.227714,1.585857,2.085050,3.226806,1.598823,3999.298566,2715.355242
7.0,60456.620121,6.297138,107.602623,14495.348514,18251.987385,15387.297859,12321.986363,0.625552,2.414655,1.450934,1.805998,10694.909870,7283.934686
8.0,36940.203080,4.403136,120.321678,4688.216472,15321.060196,5380.230181,11550.696231,0.189499,1.989927,0.497124,1.726586,10703.534811,8472.345974
9.0,26741.073843,2.677708,99.846600,3452.905479,11031.842852,3465.254782,8791.070729,0.127612,1.080766,0.272550,1.196780,7689.208622,6562.647909


In [582]:
np.sum(calmonths['Value'])/np.sum(calmonths['Energy'])*1000000

73.94447067972172